# **We do everything from scratch with 2 model pipeline vision and 0.85 acc goal**

### **Imports and config**

In [35]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedShuffleSplit, GroupShuffleSplit
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.base import clone

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

# ---------------- CONFIG ----------------
CSV_PATH = "training_data_75.csv"   # cambia a /mnt/data/... si estas en otro entorno
TEST_SIZE = 0.25
SEED = 42

LOW_MAX_THRESHOLD = 2  # low={1,2} high={>=4}

ID_COL = "file"
TARGET_COL = "min_threshold"

LEAKAGE_COLS = [
    "forward_runtime",
    "max_fidelity_achieved",
    "forward_shots",
    "forward_peak_rss_mb",
    "n_thresholds_tested",
]

NON_FEATURE_COLS = [ID_COL, "family"]  # NO eliminar backend/precision


### **1. Split**

In [37]:
"""
Split + subsets (gate/low/high)
"""

def make_gate_label(threshold_series: pd.Series, low_max: int) -> pd.Series:
    return (threshold_series.astype(int) > low_max).astype(int)  # 0=low, 1=high

def build_feature_frame(df: pd.DataFrame) -> pd.DataFrame:
    drop_cols = [TARGET_COL] + NON_FEATURE_COLS + [c for c in LEAKAGE_COLS if c in df.columns]
    drop_cols = [c for c in drop_cols if c in df.columns]
    return df.drop(columns=drop_cols).copy()

def stratified_group_holdout_split(df: pd.DataFrame, test_size=0.25, seed=42, low_max_threshold=2):
    df = df.copy()
    df[ID_COL] = df[ID_COL].astype(str)
    df[TARGET_COL] = df[TARGET_COL].astype(int)

    df["gate_label"] = make_gate_label(df[TARGET_COL], low_max_threshold)

    # file-level gate: if any row is high -> file is high
    file_table = df.groupby(ID_COL, as_index=False).agg(
        file_gate=("gate_label", "max"),
    )
    files = file_table[ID_COL].values
    strata = file_table["file_gate"].values

    if len(np.unique(strata)) >= 2 and min((strata == s).sum() for s in np.unique(strata)) >= 2:
        sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)
        tr_f, te_f = next(sss.split(files, strata))
        train_files = set(files[tr_f])
        test_files  = set(files[te_f])
        split_mode = "StratifiedShuffleSplit(file_gate)"
    else:
        gss = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)
        tr_f, te_f = next(gss.split(files, np.zeros(len(files)), groups=files))
        train_files = set(files[tr_f])
        test_files  = set(files[te_f])
        split_mode = "GroupShuffleSplit(fallback)"

    train_idx = df.index[df[ID_COL].isin(train_files)].to_numpy()
    test_idx  = df.index[df[ID_COL].isin(test_files)].to_numpy()

    X_df = build_feature_frame(df)
    y = df[TARGET_COL].to_numpy()
    groups = df[ID_COL].to_numpy()
    gate_y = df["gate_label"].to_numpy()

    out = {
        "df": df,
        "X_train": X_df.iloc[train_idx].copy(),
        "X_test":  X_df.iloc[test_idx].copy(),
        "y_train": y[train_idx].copy(),
        "y_test":  y[test_idx].copy(),
        "gate_train": gate_y[train_idx].copy(),
        "gate_test":  gate_y[test_idx].copy(),
        "groups_train": groups[train_idx].copy(),
        "groups_test":  groups[test_idx].copy(),
        "train_idx": train_idx,
        "test_idx": test_idx,
        "train_files": train_files,
        "test_files": test_files,
        "split_mode": split_mode,
    }

    assert set(out["groups_train"]).isdisjoint(set(out["groups_test"])), "Leakage: file overlap train/test!"
    print("Split mode:", split_mode)
    print("Rows:", len(df), "| Unique files:", df[ID_COL].nunique())
    print("Train rows:", len(train_idx), "| Test rows:", len(test_idx))
    print("Train files:", len(train_files), "| Test files:", len(test_files))
    return out

def make_expert_subsets(split_dict, low_max_threshold=2):
    X_train = split_dict["X_train"]
    y_train = split_dict["y_train"]
    g_train = split_dict["groups_train"]

    X_test = split_dict["X_test"]
    y_test = split_dict["y_test"]
    g_test = split_dict["groups_test"]

    # Gate labels
    gate_y_train = (y_train > low_max_threshold).astype(int)
    gate_y_test  = (y_test  > low_max_threshold).astype(int)

    # Low expert
    low_mask_tr = y_train <= low_max_threshold
    low_mask_te = y_test  <= low_max_threshold

    # High expert
    high_mask_tr = y_train > low_max_threshold
    high_mask_te = y_test  > low_max_threshold

    return {
        "gate": (X_train, gate_y_train, g_train, X_test, gate_y_test, g_test),
        "low":  (X_train[low_mask_tr], y_train[low_mask_tr], g_train[low_mask_tr],
                 X_test[low_mask_te],  y_test[low_mask_te],  g_test[low_mask_te]),
        "high": (X_train[high_mask_tr], y_train[high_mask_tr], g_train[high_mask_tr],
                 X_test[high_mask_te],  y_test[high_mask_te],  g_test[high_mask_te]),
    }

# -------- run split + subsets --------
df = pd.read_csv(CSV_PATH)
split = stratified_group_holdout_split(df, test_size=TEST_SIZE, seed=SEED, low_max_threshold=LOW_MAX_THRESHOLD)
subsets = make_expert_subsets(split, low_max_threshold=LOW_MAX_THRESHOLD)

# quick sanity
Xg_tr, yg_tr, gg_tr, *_ = subsets["gate"]
Xl_tr, yl_tr, gl_tr, *_ = subsets["low"]
Xh_tr, yh_tr, gh_tr, *_ = subsets["high"]

print("Gate train:", Xg_tr.shape, "unique files:", len(np.unique(gg_tr)))
print("Low  train:", Xl_tr.shape, "unique files:", len(np.unique(gl_tr)), "classes:", sorted(np.unique(yl_tr).tolist()))
print("High train:", Xh_tr.shape, "unique files:", len(np.unique(gh_tr)), "classes:", sorted(np.unique(yh_tr).tolist()))


Split mode: StratifiedShuffleSplit(file_gate)
Rows: 137 | Unique files: 36
Train rows: 102 | Test rows: 35
Train files: 27 | Test files: 9
Gate train: (102, 65) unique files: 27
Low  train: (76, 65) unique files: 19 classes: [1, 2]
High train: (26, 65) unique files: 8 classes: [4, 8, 16, 64]


### **Reward scoring + decision rule (max expected reward)**

In [38]:
def reward_scalar(y_true, y_pred):
    y_true = int(y_true); y_pred = int(y_pred)
    return 0.0 if y_pred < y_true else float(y_true) / float(y_pred)

def reward_matrix(classes):
    classes = np.asarray(classes, dtype=int)
    R = np.zeros((len(classes), len(classes)), dtype=float)
    for i, t in enumerate(classes):       # true
        for j, p in enumerate(classes):   # pred
            R[i, j] = reward_scalar(t, p)
    return R

def predict_max_expected_reward(proba, classes):
    classes = np.asarray(classes, dtype=int)
    R = reward_matrix(classes)            # (n_true, n_pred)
    exp_reward = proba @ R                # (n_samples, n_pred)
    best_j = np.argmax(exp_reward, axis=1)
    return classes[best_j]

def mean_reward(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=int)
    y_pred = np.asarray(y_pred, dtype=int)
    return float(np.mean([reward_scalar(t, p) for t, p in zip(y_true, y_pred)]))

GATE_FP_REWARD = 0.5  # ajustable: castigo por mandar low a high

def gate_reward_matrix():
    # classes = [0,1] (0=low, 1=high)
    # rows = true, cols = pred
    # true low(0): pred low=1, pred high=GATE_FP_REWARD
    # true high(1): pred low=0, pred high=1
    return np.array([
        [1.0, GATE_FP_REWARD],
        [0.0, 1.0]
    ], dtype=float)

def predict_gate_max_expected_reward(proba):
    # proba aligned with classes [0,1]
    R = gate_reward_matrix()
    exp_reward = proba @ R
    best = np.argmax(exp_reward, axis=1)
    return best.astype(int)



### **2. Construct 2 model pipeline based on treshold**

**Folds por grupos (file).**

In [39]:
"""
Folds por grupos (file).
"""

from collections import Counter

def _mode(arr):
    c = Counter(arr.tolist())
    return c.most_common(1)[0][0]

def make_stratified_group_folds(y, groups, n_splits=5, seed=42):
    y = np.asarray(y)
    groups = np.asarray(groups)
    unique_groups = np.unique(groups)

    # one label per group
    group_labels = np.array([_mode(y[groups == g]) for g in unique_groups])

    counts = Counter(group_labels.tolist())
    min_groups_per_class = min(counts.values())
    if n_splits > min_groups_per_class:
        n_splits = max(2, min_groups_per_class)

    print("make_stratified_group_folds: group label counts =", dict(counts), "| n_splits =", n_splits)

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

    folds = []
    for tr_g, te_g in skf.split(unique_groups, group_labels):
        tr_groups = set(unique_groups[tr_g])
        te_groups = set(unique_groups[te_g])

        tr_idx = np.where(np.isin(groups, list(tr_groups)))[0]
        te_idx = np.where(np.isin(groups, list(te_groups)))[0]

        assert set(groups[tr_idx]).isdisjoint(set(groups[te_idx]))
        folds.append((tr_idx, te_idx))

    return folds

def make_group_folds_high(y, groups, max_splits=3):
    groups = np.asarray(groups)
    n_groups = len(np.unique(groups))
    n_splits = min(max_splits, n_groups)
    n_splits = max(2, n_splits) if n_groups >= 2 else 2

    gkf = GroupKFold(n_splits=n_splits)
    X_dummy = np.zeros((len(y), 1))
    folds = [(tr, te) for tr, te in gkf.split(X_dummy, y, groups=groups)]
    return folds


**Definir Modelos y completamos el preprocesamiento.**

In [41]:
import numpy as np
from functools import partial

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

# ---------------- Feature scoring funcs ----------------
MI_SCORE = partial(mutual_info_classif, random_state=SEED)  # mas flexible pero algo ruidoso
F_SCORE  = f_classif                                       # mas estable

def build_preprocessor(X_df, k_best=None, score_func="mi"):
    """
    Preprocess:
      - OneHot categorical
      - Standardize numeric
    Optional:
      - SelectKBest after preprocessing (inside CV, no leakage)

    NOTE: sparse_threshold=0.0 forces dense output so tree models (RF/ET) wont break.
    """
    cat_cols = X_df.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
    num_cols = [c for c in X_df.columns if c not in cat_cols]

    pre = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
            ("num", StandardScaler(), num_cols),
        ],
        remainder="drop",
        sparse_threshold=0.0
    )

    if k_best is None:
        return pre

    if score_func == "mi":
        sf = MI_SCORE
    elif score_func == "f":
        sf = F_SCORE
    else:
        raise ValueError("score_func must be 'mi' or 'f'")

    return Pipeline([
        ("pre", pre),
        ("kbest", SelectKBest(score_func=sf, k=int(k_best)))
    ])

# ---------------- Models ----------------

def model_lr_binary(X_df, C=1.0, k_best=None, score_func="mi"):
    pre = build_preprocessor(X_df, k_best=k_best, score_func=score_func)
    clf = LogisticRegression(
        max_iter=12000,
        class_weight="balanced",
        solver="liblinear",
        C=float(C)
    )
    return Pipeline([("pre", pre), ("clf", clf)])

def model_lr_ovr_multiclass(X_df, C=1.0, k_best=None, score_func="mi"):
    pre = build_preprocessor(X_df, k_best=k_best, score_func=score_func)
    base = LogisticRegression(
        max_iter=12000,
        class_weight="balanced",
        solver="liblinear",
        C=float(C)
    )
    clf = OneVsRestClassifier(base)
    return Pipeline([("pre", pre), ("clf", clf)])

def model_rf(X_df, n_estimators=600, max_depth=None, min_samples_leaf=1, k_best=None, score_func="mi"):
    pre = build_preprocessor(X_df, k_best=k_best, score_func=score_func)
    clf = RandomForestClassifier(
        n_estimators=int(n_estimators),
        max_depth=max_depth,
        min_samples_leaf=int(min_samples_leaf),
        class_weight="balanced_subsample",
        random_state=SEED,
        n_jobs=-1
    )
    return Pipeline([("pre", pre), ("clf", clf)])

def model_et(X_df, n_estimators=900, max_depth=None, min_samples_leaf=1, k_best=None, score_func="mi"):
    pre = build_preprocessor(X_df, k_best=k_best, score_func=score_func)
    clf = ExtraTreesClassifier(
        n_estimators=int(n_estimators),
        max_depth=max_depth,
        min_samples_leaf=int(min_samples_leaf),
        class_weight="balanced",
        random_state=SEED,
        n_jobs=-1
    )
    return Pipeline([("pre", pre), ("clf", clf)])

def model_calibrated_linear_svc(X_df, C=1.0, k_best=None, score_func="mi"):
    pre = build_preprocessor(X_df, k_best=k_best, score_func=score_func)
    base = LinearSVC(class_weight="balanced", C=float(C), max_iter=20000)
    clf = CalibratedClassifierCV(estimator=base, method="sigmoid", cv=3)
    return Pipeline([("pre", pre), ("clf", clf)])


**Evaluador CV con reward (maneja clases ausentes en folds)**

In [42]:
def align_proba_to_classes(estimator, proba, classes_all):
    """
    estimator.classes_ gives the order of columns in proba.
    We map them into full classes_all order. Missing classes -> prob 0.
    Then renormalize to sum 1 per row (if sum>0).
    """
    classes_all = np.asarray(classes_all, dtype=int)
    est_classes = np.asarray(estimator.classes_, dtype=int)

    out = np.zeros((proba.shape[0], len(classes_all)), dtype=float)
    col_map = {c: i for i, c in enumerate(est_classes)}

    for j, c in enumerate(classes_all):
        if c in col_map:
            out[:, j] = proba[:, col_map[c]]

    row_sums = out.sum(axis=1, keepdims=True)
    # if a row sums to 0 (should not happen), leave as zeros
    mask = row_sums.squeeze() > 0
    out[mask] = out[mask] / row_sums[mask]
    return out

def eval_cv_reward_threshold_task(estimator, X, y, groups, folds, classes_ladder, name="task"):
    X = X.reset_index(drop=True)
    y = np.asarray(y, dtype=int)
    groups = np.asarray(groups)

    scores = []
    classes_ladder = np.asarray(classes_ladder, dtype=int)

    for k, (tr_idx, te_idx) in enumerate(folds, 1):
        X_tr, X_te = X.iloc[tr_idx], X.iloc[te_idx]
        y_tr, y_te = y[tr_idx], y[te_idx]

        est = clone(estimator)
        est.fit(X_tr, y_tr)

        if not hasattr(est, "predict_proba"):
            raise ValueError("Estimator must support predict_proba for expected reward decision.")

        proba = est.predict_proba(X_te)
        proba_aligned = align_proba_to_classes(est, proba, classes_ladder)

        y_pred = predict_max_expected_reward(proba_aligned, classes_ladder)
        score = mean_reward(y_te, y_pred)
        scores.append(score)

        print(f"[{name}] Fold {k} mean_reward = {score:.4f}")

    print(f"[{name}] CV mean/std reward = {np.mean(scores):.4f} / {np.std(scores):.4f}")
    return scores

def eval_cv_reward_gate_task(estimator, X, y_gate, groups, folds, name="gate"):
    """
    Gate classes are [0,1]. Use asymmetric gate reward matrix.
    """
    X = X.reset_index(drop=True)
    y_gate = np.asarray(y_gate, dtype=int)
    groups = np.asarray(groups)

    scores = []
    for k, (tr_idx, te_idx) in enumerate(folds, 1):
        X_tr, X_te = X.iloc[tr_idx], X.iloc[te_idx]
        y_tr, y_te = y_gate[tr_idx], y_gate[te_idx]

        est = clone(estimator)
        est.fit(X_tr, y_tr)

        proba = est.predict_proba(X_te)
        # align to [0,1]
        proba_aligned = align_proba_to_classes(est, proba, np.array([0,1], dtype=int))

        y_pred = predict_gate_max_expected_reward(proba_aligned)

        # gate reward per sample
        # true low(0): pred low ->1, pred high -> GATE_FP_REWARD
        # true high(1): pred low ->0, pred high ->1
        r = []
        for t, p in zip(y_te, y_pred):
            if t == 0 and p == 0: r.append(1.0)
            elif t == 0 and p == 1: r.append(GATE_FP_REWARD)
            elif t == 1 and p == 0: r.append(0.0)
            else: r.append(1.0)

        score = float(np.mean(r))
        scores.append(score)

        print(f"[{name}] Fold {k} mean_gate_reward = {score:.4f}")

    print(f"[{name}] CV mean/std gate_reward = {np.mean(scores):.4f} / {np.std(scores):.4f}")
    return scores


**Model zoo runner (prueba muchos modelos por tarea)**

In [44]:
def rank_models(scores_dict):
    rows = []
    for name, scores in scores_dict.items():
        rows.append((name, float(np.mean(scores)), float(np.std(scores))))
    rows.sort(key=lambda x: x[1], reverse=True)
    return rows

def safe_eval_gate(name, est):
    try:
        return eval_cv_reward_gate_task(est, Xg_tr, yg_tr, gg_tr, gate_folds, name=name)
    except Exception as e:
        print(f"[SKIP gate] {name} -> {type(e).__name__}: {e}")
        return None

def safe_eval_threshold(name, est, X, y, g, folds, classes):
    try:
        return eval_cv_reward_threshold_task(est, X, y, g, folds, classes, name=name)
    except Exception as e:
        print(f"[SKIP thr] {name} -> {type(e).__name__}: {e}")
        return None

# ---------------- Unpack train subsets ----------------
Xg_tr, yg_tr, gg_tr, *_ = subsets["gate"]
Xl_tr, yl_tr, gl_tr, *_ = subsets["low"]
Xh_tr, yh_tr, gh_tr, *_ = subsets["high"]

# ---------------- Folds ----------------
gate_folds = make_stratified_group_folds(yg_tr, gg_tr, n_splits=5, seed=SEED)
low_folds  = make_stratified_group_folds(yl_tr, gl_tr, n_splits=5, seed=SEED)
high_folds = make_group_folds_high(yh_tr, gh_tr, max_splits=3)

LOW_CLASSES  = np.array([1,2], dtype=int)
HIGH_CLASSES = np.array([4,8,16,64], dtype=int)

# ---------------- Grid config ----------------
K_GRID = [None, 10, 15, 20, 30]    # puedes ajustar, None = sin feature selection
SCORES = ["mi", "f"]              # mutual info vs f_classif

# ---------------- Catalog builder per stage ----------------
def build_gate_catalog(X):
    cat = {}
    for k in K_GRID:
        for sc in SCORES:
            tag = f"k{k}_{sc}"
            cat[f"gate_lr_C1_{tag}"]  = model_lr_binary(X, C=1.0, k_best=k, score_func=sc)
            cat[f"gate_lr_C3_{tag}"]  = model_lr_binary(X, C=3.0, k_best=k, score_func=sc)
            cat[f"gate_et_{tag}"]     = model_et(X, n_estimators=900, k_best=k, score_func=sc)
            cat[f"gate_rf_{tag}"]     = model_rf(X, n_estimators=600, k_best=k, score_func=sc)
    return cat

def build_low_catalog(X):
    cat = {}
    for k in K_GRID:
        for sc in SCORES:
            tag = f"k{k}_{sc}"
            cat[f"low_lr_C1_{tag}"]  = model_lr_binary(X, C=1.0, k_best=k, score_func=sc)
            cat[f"low_lr_C3_{tag}"]  = model_lr_binary(X, C=3.0, k_best=k, score_func=sc)
            cat[f"low_et_{tag}"]     = model_et(X, n_estimators=900, k_best=k, score_func=sc)
            cat[f"low_rf_{tag}"]     = model_rf(X, n_estimators=600, k_best=k, score_func=sc)
    return cat

def build_high_catalog(X):
    cat = {}
    for k in K_GRID:
        for sc in SCORES:
            tag = f"k{k}_{sc}"
            cat[f"high_lr_ovr_C1_{tag}"] = model_lr_ovr_multiclass(X, C=1.0, k_best=k, score_func=sc)
            cat[f"high_lr_ovr_C3_{tag}"] = model_lr_ovr_multiclass(X, C=3.0, k_best=k, score_func=sc)
            cat[f"high_et_{tag}"]        = model_et(X, n_estimators=1200, k_best=k, score_func=sc)
            cat[f"high_rf_{tag}"]        = model_rf(X, n_estimators=800, k_best=k, score_func=sc)
    return cat

# ---------------- Run: GATE ----------------
print("\n=== GATE FEATURE SELECTION GRID ===")
gate_models = build_gate_catalog(Xg_tr)
gate_scores = {}
for name, est in gate_models.items():
    scores = safe_eval_gate(name, est)
    if scores is not None:
        gate_scores[name] = scores

print("\nTop gate:")
for r in rank_models(gate_scores)[:10]:
    print(r)

# ---------------- Run: LOW ----------------
print("\n=== LOW FEATURE SELECTION GRID ===")
low_models = build_low_catalog(Xl_tr)
low_scores = {}
for name, est in low_models.items():
    scores = safe_eval_threshold(name, est, Xl_tr, yl_tr, gl_tr, low_folds, LOW_CLASSES)
    if scores is not None:
        low_scores[name] = scores

print("\nTop low:")
for r in rank_models(low_scores)[:10]:
    print(r)

# ---------------- Run: HIGH ----------------
print("\n=== HIGH FEATURE SELECTION GRID ===")
high_models = build_high_catalog(Xh_tr)   # <-- esta linea
high_scores = {}
for name, est in high_models.items():
    scores = safe_eval_threshold(name, est, Xh_tr, yh_tr, gh_tr, high_folds, HIGH_CLASSES)
    if scores is not None:
        high_scores[name] = scores

print("\nTop high:")
for r in rank_models(high_scores)[:10]:
    print(r)



make_stratified_group_folds: group label counts = {1: 8, 0: 19} | n_splits = 5
make_stratified_group_folds: group label counts = {1: 11, 2: 8} | n_splits = 5

=== GATE FEATURE SELECTION GRID ===
[gate_lr_C1_kNone_mi] Fold 1 mean_gate_reward = 0.9048
[gate_lr_C1_kNone_mi] Fold 2 mean_gate_reward = 0.8333
[gate_lr_C1_kNone_mi] Fold 3 mean_gate_reward = 0.8947
[gate_lr_C1_kNone_mi] Fold 4 mean_gate_reward = 1.0000
[gate_lr_C1_kNone_mi] Fold 5 mean_gate_reward = 1.0000
[gate_lr_C1_kNone_mi] CV mean/std gate_reward = 0.9266 / 0.0648
[gate_lr_C3_kNone_mi] Fold 1 mean_gate_reward = 0.9048
[gate_lr_C3_kNone_mi] Fold 2 mean_gate_reward = 0.8333
[gate_lr_C3_kNone_mi] Fold 3 mean_gate_reward = 0.8947
[gate_lr_C3_kNone_mi] Fold 4 mean_gate_reward = 1.0000
[gate_lr_C3_kNone_mi] Fold 5 mean_gate_reward = 1.0000
[gate_lr_C3_kNone_mi] CV mean/std gate_reward = 0.9266 / 0.0648
[gate_et_kNone_mi] Fold 1 mean_gate_reward = 0.9048
[gate_et_kNone_mi] Fold 2 mean_gate_reward = 1.0000
[gate_et_kNone_mi] Fold

c:\Users\Hiram\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning: Features [14 15 18 22 23 25 26 27 28 31 53 58 59] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\Hiram\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
c:\Users\Hiram\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning: Features [14 15 18 22 23 25 26 27 28 31 53 54 58 59] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\Hiram\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
c:\Users\Hiram\AppData\Loca

[gate_et_k10_f] Fold 1 mean_gate_reward = 0.8571


c:\Users\Hiram\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning: Features [14 15 18 22 23 25 26 27 28 31 53 54 58 59] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\Hiram\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


[gate_et_k10_f] Fold 2 mean_gate_reward = 0.8333


...

[high_rf_k30_f] Fold 1 mean_reward = 0.4583


c:\Users\Hiram\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning: Features [14 15 16 18 22 23 25 26 27 28 31 32 53 57 58 59 66] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\Hiram\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


[high_rf_k30_f] Fold 2 mean_reward = 0.5000


c:\Users\Hiram\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning: Features [14 15 16 18 22 23 25 26 27 28 31 32 53 57 58 59 66] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\Hiram\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


[high_rf_k30_f] Fold 3 mean_reward = 0.1667
[high_rf_k30_f] CV mean/std reward = 0.3750 / 0.1483

Top high:
('high_lr_ovr_C3_kNone_mi', 0.4583333333333333, 0.2381448361039201)
('high_lr_ovr_C3_kNone_f', 0.4583333333333333, 0.2381448361039201)
('high_lr_ovr_C3_k20_mi', 0.4583333333333333, 0.2381448361039201)
('high_lr_ovr_C3_k20_f', 0.4583333333333333, 0.2381448361039201)
('high_lr_ovr_C3_k30_mi', 0.4583333333333333, 0.2381448361039201)
('high_lr_ovr_C3_k30_f', 0.4583333333333333, 0.2381448361039201)
('high_et_k10_mi', 0.4490740740740741, 0.21286229504764165)
('high_rf_k10_mi', 0.4490740740740741, 0.21286229504764165)
('high_rf_k10_f', 0.4490740740740741, 0.21286229504764165)
('high_rf_k15_mi', 0.4490740740740741, 0.21286229504764165)


# **Hold the results accountable check overfitting on the results**

### **Holdout end-to-end evaluation**

In [ ]:
from sklearn.metrics import confusion_matrix

# ---------------- Select best configs ----------------
gate_best = model_lr_binary(Xg_tr, C=1.0, k_best=10, score_func="mi")          # gate_lr_C1_k10_mi
low_best  = model_et(Xl_tr, n_estimators=900, k_best=15, score_func="f")       # low_et_k15_f
high_best = model_lr_ovr_multiclass(Xh_tr, C=3.0, k_best=None, score_func="mi")# high_lr_ovr_C3_kNone_mi

# ---------------- Unpack holdout test subsets ----------------
Xg_tr, yg_tr, gg_tr, Xg_te, yg_te, gg_te = subsets["gate"]  # yg_* are gate labels 0/1
Xl_tr, yl_tr, gl_tr, Xl_te, yl_te, gl_te = subsets["low"]   # yl_* are thresholds {1,2}
Xh_tr, yh_tr, gh_tr, Xh_te, yh_te, gh_te = subsets["high"]  # yh_* are thresholds {4,8,16,64} (test may miss 64)

LOW_CLASSES  = np.array([1,2], dtype=int)
HIGH_CLASSES = np.array([4,8,16,64], dtype=int)

def pipeline_predict_thresholds(
    gate_model, low_model, high_model,
    X_gate_test, X_low_test, X_high_test,
    y_gate_true, y_low_true, y_high_true
):
    """
    Returns:
      - y_pred_all (len = len(X_gate_test)) thresholds predicted for each test row
      - debug_df with per-row info
    """
    # --- Gate probabilities (aligned to [0,1]) ---
    gate_proba = gate_model.predict_proba(X_gate_test)
    gate_proba = align_proba_to_classes(gate_model, gate_proba, np.array([0,1], dtype=int))

    gate_pred = predict_gate_max_expected_reward(gate_proba)  # 0=low,1=high

    # Prepare containers
    y_pred = np.zeros(len(X_gate_test), dtype=int)
    route  = np.array([""] * len(X_gate_test), dtype=object)

    # --- LOW route ---
    low_idx = np.where(gate_pred == 0)[0]
    if len(low_idx) > 0:
        # need rows from X_gate_test that correspond to low subset feature-space
        # X_low_test is already filtered to true-low, but gate may send some true-high to low
        # so we must run low_model on X_gate_test rows directly (same feature columns)
        low_proba = low_model.predict_proba(X_gate_test.iloc[low_idx])
        low_proba = align_proba_to_classes(low_model, low_proba, LOW_CLASSES)
        low_pred_thr = predict_max_expected_reward(low_proba, LOW_CLASSES)
        y_pred[low_idx] = low_pred_thr
        route[low_idx] = "LOW"

    # --- HIGH route ---
    high_idx = np.where(gate_pred == 1)[0]
    if len(high_idx) > 0:
        high_proba = high_model.predict_proba(X_gate_test.iloc[high_idx])
        high_proba = align_proba_to_classes(high_model, high_proba, HIGH_CLASSES)
        high_pred_thr = predict_max_expected_reward(high_proba, HIGH_CLASSES)
        y_pred[high_idx] = high_pred_thr
        route[high_idx] = "HIGH"

    # Build debug df
    dbg = pd.DataFrame({
        "gate_true": y_gate_true.astype(int),
        "gate_pred": gate_pred.astype(int),
        "route": route,
        "thr_true": np.where(y_gate_true==0, y_low_true, y_high_true),  # placeholder; will fix below
    })

    return y_pred, dbg

# ---------------- Train models on full train subsets ----------------
gate_best.fit(Xg_tr, yg_tr)
low_best.fit(Xl_tr, yl_tr)
high_best.fit(Xh_tr, yh_tr)

# ---------------- Prepare test "truth" arrays aligned to Xg_te rows ----------------
# Gate test set is all test rows (35). For threshold truth, we use split["y_test"].
# Easiest: get true thresholds directly from split dict.
y_true_test_threshold = split["y_test"].astype(int)
y_true_gate = split["gate_test"].astype(int)

# ---------------- Predict thresholds on full gate test frame ----------------
y_pred_test, dbg = pipeline_predict_thresholds(
    gate_best, low_best, high_best,
    Xg_te, Xl_te, Xh_te,
    y_true_gate, yl_te.astype(int), yh_te.astype(int)
)

# Fix dbg thr_true to match full test order (Xg_te order is split["test_idx"])
dbg["thr_true"] = y_true_test_threshold
dbg["thr_pred"] = y_pred_test

# ---------------- Compute final reward ----------------
rewards = np.array([reward_scalar(t, p) for t, p in zip(dbg["thr_true"], dbg["thr_pred"])], dtype=float)
dbg["reward"] = rewards

mean_total = float(np.mean(rewards))
zero_rate  = float(np.mean(rewards == 0.0))

print("\n=== HOLDOUT PIPELINE RESULTS ===")
print("Test rows:", len(dbg), "| Unique test files:", len(np.unique(split["groups_test"])))
print(f"Mean pipeline reward: {mean_total:.4f}")
print(f"Zero-rate (underestimates): {zero_rate:.4f}")

# ---------------- Gate confusion (rows=true, cols=pred) ----------------
cm = confusion_matrix(dbg["gate_true"], dbg["gate_pred"], labels=[0,1])
print("\nGate confusion matrix (true rows [low,high] x pred cols [low,high]):")
print(cm)

# Gate FN rate = true high predicted low
true_high = (dbg["gate_true"] == 1).sum()
fn = ((dbg["gate_true"] == 1) & (dbg["gate_pred"] == 0)).sum()
fp = ((dbg["gate_true"] == 0) & (dbg["gate_pred"] == 1)).sum()
true_low = (dbg["gate_true"] == 0).sum()

fn_rate = fn / true_high if true_high > 0 else 0.0
fp_rate = fp / true_low  if true_low > 0 else 0.0
print(f"\nGate FN_rate (high->low): {fn_rate:.4f}  | FP_rate (low->high): {fp_rate:.4f}")

# ---------------- Breakdown by true gate class ----------------
low_mask  = dbg["gate_true"] == 0
high_mask = dbg["gate_true"] == 1
print("\nMean reward on TRUE low:", float(dbg.loc[low_mask, "reward"].mean()) if low_mask.any() else None)
print("Mean reward on TRUE high:", float(dbg.loc[high_mask, "reward"].mean()) if high_mask.any() else None)

# ---------------- Optional: show worst cases ----------------
print("\nWorst 10 cases (reward asc):")
display(dbg.sort_values("reward").head(10))


### **4. Select Models and hiperparameter tuning [optuna]**

Step 4 and 3 can be in inverse order or iterative